* Readme


In [72]:
import os, cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import shutil

In [73]:
imgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\JPEGImages'
parentImgFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages'
xmlFileDir = r'C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_XMLAnnotations\XMLAnnotations'
directory = "CImages"


In [74]:
def extractBBox():
    os.chdir(xmlFileDir)
    imgBBoxes = []
    xmlFileNames = [f for f in listdir(xmlFileDir) if isfile(join(xmlFileDir, f))]
    #print(xmlFileNames[:10])
    for file in xmlFileNames:
        root = ET.parse(file).getroot()
        filename = root.find('filename').text
        xmin = int(root.find('object/bndbox/xmin').text)
        ymin = int(root.find('object/bndbox/ymin').text)
        xmax = int(root.find('object/bndbox/xmax').text)
        ymax = int(root.find('object/bndbox/ymax').text)
    
        imgBBoxes.append([filename, xmin, ymin, xmax, ymax])
    print("Done")
    return imgBBoxes
BBoxes = extractBBox()

Done


In [75]:
def cropImages(BBoxes):
    #make cropped images folder
    path = os.path.join(parentImgFileDir, directory)
    print(path+'/')
    if os.path.exists(path):
        print("it exist")
    else:
        print("does not exist, create folder")
        os.mkdir(path)
    
    os.chdir(imgFileDir)
    count = 0
    for image in BBoxes:
        #print("cropping: ", image[0])
        print(str(count)+"/"+str(len(BBoxes)), end='\r')
        count +=1
        im =Image.open(image[0])
        xmin = image[1]
        ymin = image[2]
        xmax = image[3]
        ymax = image[4]
        cImg = im.crop((xmin, ymin, xmax,ymax))
        cImg.save(path+'/'+image[0], 'JPEG')
    print()
    print("Done")
cropImages(BBoxes)

C:\Users\WillyB\Documents\GitHub\embedded_project\Stanford40_JPEGImages\CImages/
does not exist, create folder
9531/9532
Done


In [83]:
def moveImgTrainTest():
    #make cropped images folder
    croppedImgPath = os.path.join(parentImgFileDir, directory)
    os.chdir(parentImgFileDir)

    #create Test and Train folders
    train_path = os.path.join(parentImgFileDir, "Train")
    test_path = os.path.join(parentImgFileDir, "Test")
    if os.path.exists(train_path):
        print("Train folder already exists")
    else:
        print("Train folder does not exist, create folder")
        os.mkdir(train_path)
    if os.path.exists(test_path):
        print("Test folder already exists")
    else:
        print("Test folder does not exist, create folder")
        os.mkdir(test_path)
    trainlist = [line.rstrip('\n') for line in open('train.txt', 'r')]
    #testlist = open('test.txt', 'r').readlines()
    testlist = [line.rstrip('\n') for line in open('test.txt', 'r')]
    
    #Copy Test and Train images to the correct folder
    os.chdir(croppedImgPath)
    test_count = 0
    train_count = 0
    for img in trainlist:
        #print(str(test_count)+'/'+str(len(trainlist)), end='\r')
        shutil.copy(img, train_path)
        train_count += 1
    print()
    print("moved all train images", train_count)
    
    for img in testlist:
        shutil.copy(img, test_path)
        #print(str(train_count)+'/'+str(len(testlist)), end='\r')
        test_count += 1
    print()
    print("moved all test images", test_count)


moveImgTrainTest()


Train folder already exists
Test folder already exists
4000
['applauding_177.jpg', 'applauding_119.jpg']
5532
['applauding_254.jpg', 'applauding_073.jpg']

moved all train images 4000

moved all test images 5532
